## Import Libraries

In [1]:
# You may need to install some of the libraries below
# If so, uncomment any of the below commands
# !pip install matplotlib
# !pip install seaborn
# !pip install mplcyberpunk
# !pip install scikit-learn
# !pip install pickle

In [2]:
import pandas as pd
import regex as re
import numpy as np
import logging

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
%config InlineBackend.figure_formats = ['svg']

# ML Modeling
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay,roc_curve
from sklearn.preprocessing import normalize
# Saving and importing trained models
import pickle

## 2. Import and Review the Data

In [20]:
# Imports the noted CSV file as a pandas DataFrame based on the path below
df = pd.read_csv("data/original-dataset.csv", delimiter=",", encoding='utf-8')

/var/folders/rj/l_96jjr56y39j9jw2hnmv_wh0000gn/T/ipykernel_1694/617997427.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/original-dataset.csv", delimiter=",", encoding='utf-8')


### 2.1 General shape and content of the dataset

In [21]:
df.shape

(153434, 18)

In [22]:
df.head(5)

,ARREST BOOKING NO_PK,GO NO,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST_BLOCK,ARREST REASON,ARREST TYPE,BEAT,SUMMARY OF FACTS,ARREST OFFICER,officer_name,CURRENT STATUS,YOUNG OFFENDER
0,SJ20168067,SJ2016161030552,441897259,33.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1425.0,ROBERTS AV / PHELAN AV,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
1,SJ20168127,SJ2016161030674,212271361,27.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1701.0,1300 BLOCK SENTER RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
2,SJ20168067,SJ2016J3297232 C,441897259,33.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1425.0,ROBERTS AV / PHELAN AV,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
3,SJ20168127,SJ2016J3297233 C,212271361,27.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1701.0,1300 BLOCK SENTER RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
4,SJ20153325,NaN,441844005,52.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,02/25/15,1500.0,ROBERTS AV / STORY RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N


In [23]:
df.describe()

,PIN,AGE,ARREST TIME
count,1.534340e+05,153352.000000,153422.000000
mean,4.050585e+08,34.969267,1375.358554
std,9.573970e+07,12.717352,700.107685
min,6.579400e+04,5.000000,0.000000
25%,4.417536e+08,25.000000,935.000000
50%,4.418847e+08,33.000000,1523.000000
75%,4.420149e+08,44.000000,1930.000000
max,4.422767e+08,95.000000,2359.000000


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153434 entries, 0 to 153433
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ARREST BOOKING NO_PK      153434 non-null  object 
 1   GO NO                     124373 non-null  object 
 2   PIN                       153434 non-null  int64  
 3   AGE                       153352 non-null  float64
 4   SEX                       153347 non-null  object 
 5   RACE                      153433 non-null  object 
 6   ETHNICITY                 153272 non-null  object 
 7   ARREST DATE               153434 non-null  object 
 8   ARREST TIME               153422 non-null  float64
 9   LOCATION OF ARREST_BLOCK  153434 non-null  object 
 10  ARREST REASON             153404 non-null  object 
 11  ARREST TYPE               153434 non-null  object 
 12  BEAT                      137510 non-null  object 
 13  SUMMARY OF FACTS          153434 non-null  o

## 3. Summary Statistics &amp; Data Processing

In [27]:
df.describe(include='all')

,ARREST BOOKING NO_PK,GO NO,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST_BLOCK,ARREST REASON,ARREST TYPE,BEAT,SUMMARY OF FACTS,ARREST OFFICER,officer_name,CURRENT STATUS,YOUNG OFFENDER
count,153434,124373,1.534340e+05,153352.000000,153347,153433,153272,153434,153422.000000,153434,153404,153434,137510,153434,104192,104140,153434,153434
unique,124395,108326,NaN,NaN,3,19,19,2100,NaN,22074,11,2,88,8185,1403,1108,3,2
top,SJ20175880,SJ2018WARRANT C,NaN,NaN,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,06/21/19,NaN,NO LOCATION INCLUDED IN RECORD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L4,POSSESS UNLAWFUL PARAPHERNALIA (CITE),2869,GUERRA,CHARGED/BOOKED,N
freq,15,824,NaN,NaN,117072,83551,83360,150,NaN,8424,75193,95097,5411,7146,1202,1250,76304,145293
mean,NaN,NaN,4.050585e+08,34.969267,NaN,NaN,NaN,NaN,1375.358554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,9.573970e+07,12.717352,NaN,NaN,NaN,NaN,700.107685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,6.579400e+04,5.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.417536e+08,25.000000,NaN,NaN,NaN,NaN,935.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,4.418847e+08,33.000000,NaN,NaN,NaN,NaN,1523.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,4.420149e+08,44.000000,NaN,NaN,NaN,NaN,1930.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.1 Rename Columns

In [28]:
df = df.rename(columns={'ARREST BOOKING NO_PK': 'ARREST UNIQUE ID', 'GO NO': 'INCIDENT NUMBER', 'officer_name': 'OFFICER NAME', 'LOCATION OF ARREST_BLOCK': 'LOCATION OF ARREST'})

In [29]:
df.head()

,ARREST UNIQUE ID,INCIDENT NUMBER,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST,ARREST REASON,ARREST TYPE,BEAT,SUMMARY OF FACTS,ARREST OFFICER,OFFICER NAME,CURRENT STATUS,YOUNG OFFENDER
0,SJ20168067,SJ2016161030552,441897259,33.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1425.0,ROBERTS AV / PHELAN AV,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
1,SJ20168127,SJ2016161030674,212271361,27.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1701.0,1300 BLOCK SENTER RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
2,SJ20168067,SJ2016J3297232 C,441897259,33.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1425.0,ROBERTS AV / PHELAN AV,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
3,SJ20168127,SJ2016J3297233 C,212271361,27.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/12/16,1701.0,1300 BLOCK SENTER RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N
4,SJ20153325,NaN,441844005,52.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,02/25/15,1500.0,ROBERTS AV / STORY RD,CRIMINAL CITATION,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,L2,SJ-TRESP ON CITY PROPERTY WHEN CLOSED UPON ORD...,057P,ABE,CITED,N


### 3.2 Drop Columns

In [30]:
df = df.drop(columns='BEAT')

### 3.3 Dealing with missing data .isna() / .notna()

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153434 entries, 0 to 153433
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ARREST UNIQUE ID    153434 non-null  object 
 1   INCIDENT NUMBER     124373 non-null  object 
 2   PIN                 153434 non-null  int64  
 3   AGE                 153352 non-null  float64
 4   SEX                 153347 non-null  object 
 5   RACE                153433 non-null  object 
 6   ETHNICITY           153272 non-null  object 
 7   ARREST DATE         153434 non-null  object 
 8   ARREST TIME         153422 non-null  float64
 9   LOCATION OF ARREST  153434 non-null  object 
 10  ARREST REASON       153404 non-null  object 
 11  ARREST TYPE         153434 non-null  object 
 12  SUMMARY OF FACTS    153434 non-null  object 
 13  ARREST OFFICER      104192 non-null  object 
 14  OFFICER NAME        104140 non-null  object 
 15  CURRENT STATUS      153434 non-nul

Notice how there are lots of missing values: of the total 153434 rows, the arrest reason' columns only have 153404 values.

Let's first examine this to better understand it.

### Check NULL in the datasets

In [32]:
df.isnull().sum()

ARREST UNIQUE ID          0
INCIDENT NUMBER       29061
PIN                       0
AGE                      82
SEX                      87
RACE                      1
ETHNICITY               162
ARREST DATE               0
ARREST TIME              12
LOCATION OF ARREST        0
ARREST REASON            30
ARREST TYPE               0
SUMMARY OF FACTS          0
ARREST OFFICER        49242
OFFICER NAME          49294
CURRENT STATUS            0
YOUNG OFFENDER            0
dtype: int64

In [35]:
df[df['ARREST REASON'].isna()].head(5)

,ARREST UNIQUE ID,INCIDENT NUMBER,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST,ARREST REASON,ARREST TYPE,SUMMARY OF FACTS,ARREST OFFICER,OFFICER NAME,CURRENT STATUS,YOUNG OFFENDER
1659,SJ20205424,SJ2020200380457,223478286,59.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,02/07/20,1500.0,400 BLOCK N 1ST ST,NaN,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,CONFINEMENT PENDING HEARING (REMB),4596,ALVAREZ,CHARGED/BOOKED,N
3611,SJ20182042,NaN,441779324,41.0,F,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,01/31/18,1059.0,1900 BLOCK LAS PLUMAS AV,NaN,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,CONFINEMENT PENDING DECISION ON REQUEST FOR RE...,3233,ARDIZZONE,CHARGED/BOOKED,N
24242,SJ20208893,NaN,442254649,42.0,M,CAUCASIAN,CAUCASIAN,06/04/20,1315.0,400 BLOCK N 1ST ST,NaN,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,CONFINEMENT PENDING HEARING (REMB),4635,DEGRAVE,CHARGED/BOOKED,N
25008,SJ20208422,SJ2020201200869,442254313,60.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,04/29/20,2010.0,100 BLOCK N 33RD ST,NaN,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,CONFINEMENT PENDING HEARING (REMB),3946,DELOSSANTOS,CHARGED/BOOKED,N
28998,SJ201813206,NaN,163578113,46.0,M,HISPANIC/LATIN/MEXICAN,HISPANIC/LATIN/MEXICAN,08/02/18,1011.0,900 BLOCK LOCUST ST,NaN,TAKEN INTO CUSTODY/WARRANT ARREST ONLY,INFLICT CORPORAL INJ SPOUSE/COHAB (REMB),4613,ESPINO,CHARGED/BOOKED,N


This is important information for the sake of better understanding our dataset. But it's also important because `NaN` values are treated as *floats*, not *strings*. If we tried to manipulate this column as text data, we would get an error. For this reason, we're going to replace or "fill" these `NaN` values with the string "No Arrest Reason" by using the `.fillna()` method.

In [36]:
df['ARREST REASON'] = df['ARREST REASON'].fillna('No Arrest Reason')

In [37]:
df[df['ARREST REASON'].isna()]

,ARREST UNIQUE ID,INCIDENT NUMBER,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST,ARREST REASON,ARREST TYPE,SUMMARY OF FACTS,ARREST OFFICER,OFFICER NAME,CURRENT STATUS,YOUNG OFFENDER


Repeat this Process For The Other Columns

In [41]:
df['INCIDENT NUMBER'] = df['INCIDENT NUMBER'].fillna('No Incident Number')
df['AGE'] = df['AGE'].fillna('No Age Data')
df['SEX'] = df['SEX'].fillna('No Sex Data')
df['RACE'] = df['RACE'].fillna('No Race Data')
df['ETHNICITY'] = df['ETHNICITY'].fillna('No Ethnicity Data')
df['ARREST TIME'] = df['ARREST TIME'].fillna('No Time Data')
df['ARREST OFFICER'] = df['ARREST OFFICER'].fillna('No Officer Data')
df['OFFICER NAME'] = df['OFFICER NAME'].fillna('No Officer Data')

df.isnull().sum()

ARREST UNIQUE ID      0
INCIDENT NUMBER       0
PIN                   0
AGE                   0
SEX                   0
RACE                  0
ETHNICITY             0
ARREST DATE           0
ARREST TIME           0
LOCATION OF ARREST    0
ARREST REASON         0
ARREST TYPE           0
SUMMARY OF FACTS      0
ARREST OFFICER        0
OFFICER NAME          0
CURRENT STATUS        0
YOUNG OFFENDER        0
dtype: int64

### 3.4 Check for Duplicates 

In [42]:
df.duplicated(keep=False)

0         False
1         False
2         False
3         False
4         False
          ...  
153429    False
153430    False
153431    False
153432    False
153433    False
Length: 153434, dtype: bool

In [43]:
df[df.duplicated(keep=False)]

,ARREST UNIQUE ID,INCIDENT NUMBER,PIN,AGE,SEX,RACE,ETHNICITY,ARREST DATE,ARREST TIME,LOCATION OF ARREST,ARREST REASON,ARREST TYPE,SUMMARY OF FACTS,ARREST OFFICER,OFFICER NAME,CURRENT STATUS,YOUNG OFFENDER


## 11. Write to CSV

In [44]:
df.to_csv("data/processed-dataset.csv", encoding='utf-8', index=False)